In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.tf_adam_optimizer import AdamOptimizer
from ECD_control.gate_sets.SNAP_disp import SNAP_disp
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

2022-02-16 16:56:24.810337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-16 16:56:24.810350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



In [2]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 20
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'beta_scale' : 3.0
}
SNAP_gate_set = SNAP_disp(**gate_set_params)

2022-02-16 16:56:25.676449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-16 16:56:25.676465: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-16 16:56:25.676476: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sauron): /proc/driver/nvidia/version does not exist
2022-02-16 16:56:25.676629: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#The target oscillator state.
fock = 1
psi_t = qt.basis(N,fock) #target state

In [4]:
#the optimization options
synth_params = {
    'gateset' : SNAP_gate_set,
    'N_blocks':6,
    'N_multistart' : 100, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [5]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

ValueError: in user code:

    File "/home/rsl/ECD_control/ECD_control/ECD_optimization/GateSynthesizer.py", line 174, in batch_state_transfer_fidelities_real_part  *
        bs = self.gateset.batch_construct_block_operators(opt_params)
    File "/home/rsl/ECD_control/ECD_control/gate_sets/SNAP_disp.py", line 64, in batch_construct_block_operators  *
        blocks = Ds @ SNAPs

    ValueError: Dimensions must be equal, but are 20 and 100 for '{{node matmul}} = BatchMatMulV2[T=DT_COMPLEX64, adj_x=false, adj_y=false](PartitionedCall, StatefulPartitionedCall)' with input shapes: [6,100,20,20], [6,100,100].


In [6]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-02-16 15:16:38
 Epoch: 8 / 100 Max Fid: 0.998438 Avg Fid: 0.778060 Max dFid: 0.118635 Avg dFid: 0.054608 Elapsed time: 0:00:12.966818 Remaing time: 0:02:29.118411

 Optimization stopped. Term fidelity reached.

filename: Fock 1.h5

Best circuit parameters found:
betastf.Tensor(
[ 1.13286-0.70326j  0.99961+0.20127j  0.96816-0.80636j -0.58972-0.57687j
 -0.04752+0.83982j  0.05504-0.04376j], shape=(6,), dtype=complex64)
phistf.Tensor([-0.6859   0.15728 -0.29933  0.05631 -2.32845 -1.64227], shape=(6,), dtype=float32)
etastf.Tensor([ 2.13243  2.27639  0.18835  1.06176 -1.46157 -1.68345], shape=(6,), dtype=float32)
thetastf.Tensor([ 2.03892 -2.89281  2.66991  3.09822  2.86506  1.59467], shape=(6,), dtype=float32)


all data saved as: Fock 1.h5
termination reason: term_fid
optimization timestamp (start time): 2022-02-16 15:16:38
timestamp (end time): 2022-02-16 15:16:51
elapsed time: 0:00:12.970937
Time per epoch (epoch size = 10): 0:00:01.621367
Time per Adam step (N_multista

'2022-02-16 15:16:38'

In [7]:
#can print info, including the best circuit found.
gatesynth.print_info() 

filename: Fock 1.h5

Best circuit parameters found:
betastf.Tensor(
[ 0.96794+0.48567j -0.75877+0.5005j   0.49979+0.66834j  0.50006-0.01919j
  0.03466+0.0955j  -0.10866-0.02954j], shape=(6,), dtype=complex64)
phistf.Tensor([ 1.80369  2.24556 -1.11736 -0.21043  1.80533 -2.30418], shape=(6,), dtype=float32)
etastf.Tensor([-2.36898  0.7659   2.0391   2.0377   0.00534 -0.96416], shape=(6,), dtype=float32)
thetastf.Tensor([-2.86792 -2.40265 -2.69464  1.99207 -0.44992  1.91098], shape=(6,), dtype=float32)




In [34]:
print(gatesynth.best_fidelity())

0.99766856


In [35]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)

{'betas': <tf.Tensor: shape=(6,), dtype=complex64, numpy=
array([-1.1746151 -0.8693623j , -0.49395102+0.5756166j ,
       -0.79899365+0.05219852j,  0.16476296+0.29228175j,
       -0.1956888 -0.38825154j,  0.03830615-0.18385214j], dtype=complex64)>, 'phis': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-2.4586654,  1.3162277,  3.1164181,  1.0341647,  1.1115539,
       -2.9524856], dtype=float32)>, 'etas': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 1.4823964,  2.5061128,  2.1089766,  1.0687244,  2.3668997,
       -2.2753816], dtype=float32)>, 'thetas': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-1.5192764 ,  1.4105847 , -2.4111152 , -1.9639561 ,  2.6869357 ,
       -0.73564196], dtype=float32)>}
